In [16]:
import numpy as np
import pandas as pd



In [17]:
#reading excel file
sheet = pd.read_excel('cik_list.xlsx',sheet_name = 'cik_list_ajay')

In [18]:
sheet.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [19]:
#add the url
sheet['SECFNAME'] =   'https://www.sec.gov/Archives/' + sheet['SECFNAME']

In [20]:
sheet.tail(20)

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
132,6260,ANACOMP INC,199812,1998-12-29,10-K405,https://www.sec.gov/Archives/edgar/data/6260/0...
133,6260,ANACOMP INC,199902,1999-02-11,10-Q,https://www.sec.gov/Archives/edgar/data/6260/0...
134,6260,ANACOMP INC,199905,1999-05-17,10-Q,https://www.sec.gov/Archives/edgar/data/6260/0...
135,6260,ANACOMP INC,199908,1999-08-13,10-Q,https://www.sec.gov/Archives/edgar/data/6260/0...
136,11860,BETHLEHEM STEEL CORP /DE/,199811,1998-11-12,10-Q,https://www.sec.gov/Archives/edgar/data/11860/...
137,11860,BETHLEHEM STEEL CORP /DE/,199903,1999-03-24,10-K405,https://www.sec.gov/Archives/edgar/data/11860/...
138,11860,BETHLEHEM STEEL CORP /DE/,199905,1999-05-14,10-Q,https://www.sec.gov/Archives/edgar/data/11860/...
139,11860,BETHLEHEM STEEL CORP /DE/,199906,1999-06-29,10-K/A,https://www.sec.gov/Archives/edgar/data/11860/...
140,11860,BETHLEHEM STEEL CORP /DE/,199908,1999-08-05,10-Q,https://www.sec.gov/Archives/edgar/data/11860/...
141,11860,BETHLEHEM STEEL CORP /DE/,199911,1999-11-02,10-Q,https://www.sec.gov/Archives/edgar/data/11860/...


In [21]:
import requests

In [22]:
#using requests to load text from urls
urls = sheet.loc[1:,"SECFNAME"].copy()

In [24]:
for i in urls.index:
    response = requests.get(urls[i])
    data = response.text
    urls[i] = data

In [25]:
urls.dtype

dtype('O')

In [26]:
#the strings have a lot text related to filing which is not required and is consistent in all files, splitting and getting rid of the text
#it will help later as we will essentially have html text left in case of html files
head_split = urls.str.split("<[Tt][Ee][Xx][Tt]>",1,expand=True)

In [27]:
reports = head_split[1]

In [28]:
reports.head()

1    \n\n\n- --------------------------------------...
2    \n\n\n\n                     SECURITIES AND EX...
3    \n\n==========================================...
4    \n\n\n\n                     SECURITIES AND EX...
5    \n\n\n========================================...
Name: 1, dtype: object

In [29]:
#next step is to separate strings into the ones containing markup and the ones with no markup, here its html from looking at data

b4 = reports.str.contains("HTML", case=False, regex=False)
    

In [30]:
count =1
for i in b4:
    if i == True:
        count = count+1
print(count)

53


In [31]:
html_reports = reports[b4]

In [32]:
html_reports.shape

(52,)

In [33]:
txt_reports = reports[~b4]

In [34]:
txt_reports.head()

1    \n\n\n- --------------------------------------...
2    \n\n\n\n                     SECURITIES AND EX...
3    \n\n==========================================...
4    \n\n\n\n                     SECURITIES AND EX...
5    \n\n\n========================================...
Name: 1, dtype: object

In [35]:
# now it's easier to get rid of html tags using BeautifulSoup
from bs4 import BeautifulSoup


In [36]:
#getting text from html , also using unicode.normalize to get rid of /xa0 numbers for space
import unicodedata
for i in html_reports.index:
    soup = BeautifulSoup(html_reports[i])
    for script in soup(["script", "style"]):
        script.extract()    
    html_reports[i] = unicodedata.normalize("NFKD",soup.get_text()) 
    

# now we both kinds in text with some minor additions in html cleaned files
# now I get the files with sections present and get variables with only those files for all three sections


In [81]:
b1 = html_reports.str.contains("MANAGEMENT'S DISCUSSION AND ANALYSIS", case=False, regex=False)

count =0
for i in b1:
    if i == True:
        count = count+1
print(count)

b2 = txt_reports.str.contains("MANAGEMENT'S DISCUSSION AND ANALYSIS", case=False, regex=False)

count = 0
for i in b2:
    if i == True:
        count = count+1
print(count)

# we can see that the section Management's discussion and analysis is a part of 90 reports in total. We can say that with certainty because the searches here as case-sensitive which means these are headings

bq = html_reports.str.contains("QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK", case=False, regex=False)

count =0
for i in bq:
    if i == True:
        count = count+1
print(count)

bq2 = txt_reports.str.contains("QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK", case=False, regex=False)
count =0
for i in bq2:
    if i == True:
        count = count+1
print(count)

#similary for quantitative and qualitative ...  there are 44 in total

br = html_reports.str.contains("RISK FACTORS", case=False, regex=False)
count=0
for i in br:
    if i == True:
        count = count+1
print(count)


br2 = txt_reports.str.contains("RISK FACTORS", case=False, regex=False)
count=0
for i in br2:
    if i == True:
        count = count+1
print(count)

#there are 32 documents with risk factors. only in the ones with html

#now report with respective section occurences only, so they can be extracted only, index from original preserved
mdah = html_reports[b1]
mdat = txt_reports[b2]
qqah =  html_reports[bq]
qqat =  txt_reports[bq2]
rfh = html_reports[br]
rft = txt_reports[br2]

15
88
30
34
35
4


In [38]:
#original index has been preserved. now I split the mda files  on the title and observe
x = mdah.str.split("Management's Discussion and Analysis",expand=True)

In [39]:
# we see that there are 2 or three mentions of the title, we select the last part of each file's splits
mdahx = []
for i in x.index:
    if x.loc[i,3] is not None:
        mdahx.append(x.loc[i,3])
    elif x.loc[i,2] is not None:
        mdahx.append(x.loc[i,2])
    else:
        mdahx.append(x.loc[i,1])

In [40]:
mdahx = pd.Series(mdahx,index=x.index)
mdahx

53      of Financial Condition and Results of\nOperat...
54      of Financial Condition and\nResults of Operat...
55      of Financial Condition\nand Results of Operat...
56      of Financial Condition\nand Results of Operat...
57      of Financial Condition and Results of Operati...
59      of Financial Condition and Results of Operati...
60      of Financial Condition and Results of Operati...
113     of Financial Condition and Results of Operati...
116     of Financial Condition and\nResults of Operat...
117     of Financial Condition and\nResults of Operat...
118     of Financial Condition\nand Results of Operat...
119     of Financial Condition and Results\nof Operat...
120     of Financial Condition and Results of Operati...
122     of Financial Condition and Results of Operati...
123     of Financial Condition and Results of Operati...
dtype: object

In [50]:
#now for text files
y = mdat.str.split("Management's Discussion and Analysis",expand=True)
# we see that there are 2 or three mentions of the title, we select the last part of each file's splits
mdaty = []
for i in y.index:
    if y.loc[i,3] is not None:
        mdaty.append(y.loc[i,3])
    elif (y.loc[i,2] is not None) or i==73:
        mdaty.append(y.loc[i,2])
    else:
        mdaty.append(y.loc[i,1])
mdaty = pd.Series(mdaty,index=y.index)
mdaty

1       of Financial Condition and\n                 ...
5       of Financial\nCondition and Results of Operat...
6       of Financial Condition and\nResults of Operat...
7       of Financial Condition and\nResults of Operat...
9       of Financial Condition and Results of\nOperat...
11      of Financial Condition\n                    a...
13      of Financial Condition\n                    a...
14     ;\n\n\n                                       ...
15      of Financial Condition\n                     ...
16      of Financial Condition\n                    a...
18      of Financial Condition and Results of\nOperat...
43      of Financial Condition and\n         Results ...
44      of Financial Condition and\n         Results ...
45      of Financial Condition and Results of Operati...
46      of Financial Condition\n         and Results ...
47      of Financial Condition\n         and Results ...
48      of Financial Condition\n         and Results ...
51      of Financial Condition\

In [77]:
#splitting mdahx and mdaty on item to get the section
bx = txt_reports.str.contains("MANAGEMENT'S DISCUSSION AND ANALYSIS", case=True, regex=False)

count =0
for i in bx:
    if i == True:
        count = count+1
print(count)


65


In [71]:
mdaty.str.split("[Ii][Tt][Ee][Mm]",expand=True).loc[1,8]

" 2. MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS\n        OF OPERATIONS\n\n     The following discussion should be read in conjunction with the\naccompanying condensed consolidated financial statements for the three months\nended March 31, 1998 and March 30, 1997.\n\n     Net sales for the three months ended March 31, 1998 were $244.3 million, a\ndecrease of $9.2 million (3.6%) from the comparable period of the prior year.\nFirst quarter 1998 sales included $14.8 million of Coleman's sales from the\ncompletion of Step One of the Coleman acquisition on March 30, 1998 as further\ndescribed below and in Note 2 to the condensed consolidated financial\nstatements. Sales for the first quarter of 1997 included approximately $14.4\nmillion from certain divested product lines (time and temperature products and\nCounselor/registered trademark/ and Borg/registered trademark/ branded scales)\nand inventory liquidations from the Company's 1996 SKU rationalization program

In [ ]:
#we see that there is only one row with more splits than 3

In [582]:
sheet.loc[73,"SECFNAME"]

'-----BEGIN PRIVACY-ENHANCED MESSAGE-----\nProc-Type: 2001,MIC-CLEAR\nOriginator-Name: webmaster@www.sec.gov\nOriginator-Key-Asymmetric:\n MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen\n TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB\nMIC-Info: RSA-MD5,RSA,\n F4zRmzY9txflkNaunaOvFWRf6KSK6t0P8crUnfV+RLQ4u2IH/pArmNGM4E5MT2s/\n s7XPG8JW/LtrcbhcRx0Ywg==\n\n<SEC-DOCUMENT>0000005588-98-000008.txt : 19981116\n<SEC-HEADER>0000005588-98-000008.hdr.sgml : 19981116\nACCESSION NUMBER:\t\t0000005588-98-000008\nCONFORMED SUBMISSION TYPE:\t10-Q\nPUBLIC DOCUMENT COUNT:\t\t4\nCONFORMED PERIOD OF REPORT:\t19980930\nFILED AS OF DATE:\t\t19981113\n\nFILER:\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tAMERICAN PAD & PAPER CO\n\t\tCENTRAL INDEX KEY:\t\t\t0000005588\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tCONVERTED PAPER & PAPERBOARD PRODS (NO CONTAINERS/BOXES) [2670]\n\t\tIRS NUMBER:\t\t\t\t043164298\n\t\tSTATE OF INCORPORATION:\t\t\tDE\n\t\tFISCAL YEAR END:\t\t\t123